# Decoding with SpaceNet  
https://nilearn.github.io/modules/generated/nilearn.decoding.SpaceNetClassifier.html#nilearn.decoding.SpaceNetClassifier

In [ ]:
import numpy as np
import pandas as pd
import os
import numpy as np
import nilearn
import glob
from nilearn.image import index_img
#from nilearn.datasets import load_mni152_brain_mask


### Load the Chocolate data 
#### Paradigm: h2O, unapp, app
- Starting with W1 (133 Subjects)

In [ ]:
#mask_img = load_mni152_brain_mask()
mask_img="/projects/niblab/nilearn_projects/power_roimask_4bi.nii.gz"
stim = os.path.join('/projects','niblab','scripts','nilean_stuff','label_all_sub.csv')
behavioral = pd.read_csv(stim, sep=",")
conditions =  behavioral['label']
condition_mask = conditions.isin(['unapp', 'app', 'H2O'])


# Split data into train and test samples, using the chunks
condition_mask_train = (condition_mask) & (behavioral['sub'] == 0)
condition_mask_test = (condition_mask) & (behavioral['sub']  == 1)


# Apply this sample mask to X (fMRI data) and y (behavioral labels)
# Because the data is in one single large 4D image, we need to use
# index_img to do the split easily
func_filenames='/projects/niblab/bids_projects/Experiments/ChocoData/derivatives/higher_level_ana/w1_imagine_all.nii.gz'
X_train = index_img(func_filenames, condition_mask_train)
X_test = index_img(func_filenames, condition_mask_test)
y_train = conditions[condition_mask_train]
y_test = conditions[condition_mask_test]

# Compute the mean epi to be used for the background of the plotting
from nilearn.image import mean_img
background_img = mean_img(func_filenames)

### Fit SpaceNet with a Graph-Net penalty
SpaceNetClassifier 
    - adding mask
    - need to look at alphas and cv

In [ ]:
from nilearn.decoding import SpaceNetClassifier
# Fit model on train data and predict on test data
decoder = SpaceNetClassifier(memory_level=2, n_jobs=4,
                             mask=mask_img, standardize=True,
                             memory="nilearn_cache", penalty='graph-net')
decoder.fit(X_train, y_train)
y_pred = decoder.predict(X_test)
accuracy = (y_pred == y_test).mean() * 100.
print("Graph-net classification accuracy : %g%%" % accuracy)

    [NiftiMasker.fit] Loading data from None
    [NiftiMasker.fit] Resampling mask
    [Parallel(n_jobs=4)]: Done  24 out of  24 | elapsed:   36.3s finished
    Time Elapsed: 53.6853 seconds, 0 minutes.
    Graph-net classification accuracy : 35.5416%

In [ ]:
from nilearn.plotting import plot_stat_map
coef_img = decoder.coef_img_
img = plot_stat_map(coef_img, background_img,
              title="graph-net: accuracy %g%%" % accuracy)
#coef_img.to_filename('choco_graph-net_weights.nii')
img.savefig("choco_imagine_graph-net.png")

### Now Fit SpaceNet with a TV-l1 penalty

In [ ]:
decoder = SpaceNetClassifier(memory_level=2, n_jobs=4,
                             mask=mask_img, standardize=True,
                             memory="nilearn_cache", penalty='tv-l1')
decoder.fit(X_train, y_train)
y_pred = decoder.predict(X_test)
accuracy = (y_pred == y_test).mean() * 100.
print("TV-l1 classification accuracy : %g%%" % accuracy)

    [NiftiMasker.fit] Loading data from None  
    [NiftiMasker.fit] Resampling mask  
    [Parallel(n_jobs=4)]: Done  24 out of  24 | elapsed:   34.7s finished  
    Time Elapsed: 52.2088 seconds, 0 minutes.  
    TV-l1 classification accuracy : 35.2977%

## PARADIGM: app, h2O

- W1 (133 subjects) 

In [ ]:
#mask_img = load_mni152_brain_mask()
mask_img="/projects/niblab/nilearn_projects/power_roimask_4bi.nii.gz"
stim = os.path.join('/projects','niblab','scripts','nilean_stuff','label_all_sub.csv')
behavioral = pd.read_csv(stim, sep=",")
conditions =  behavioral['label']
condition_mask = conditions.isin(['app', 'H2O'])


# Split data into train and test samples, using the chunks
condition_mask_train = (condition_mask) & (behavioral['sub'] == 0)
condition_mask_test = (condition_mask) & (behavioral['sub']  == 1)


# Apply this sample mask to X (fMRI data) and y (behavioral labels)
# Because the data is in one single large 4D image, we need to use
# index_img to do the split easily
func_filenames='/projects/niblab/bids_projects/Experiments/ChocoData/derivatives/group_ana/w1_imagine_all.nii.gz'
X_train = index_img(func_filenames, condition_mask_train)
X_test = index_img(func_filenames, condition_mask_test)
y_train = conditions[condition_mask_train]
y_test = conditions[condition_mask_test]

# Compute the mean epi to be used for the background of the plotting
from nilearn.image import mean_img
background_img = mean_img(func_filenames)

In [ ]:
from nilearn.decoding import SpaceNetClassifier
# Fit model on train data and predict on test data
decoder = SpaceNetClassifier(memory_level=2, n_jobs=4,
                             mask=mask_img, standardize=True,
                             memory="nilearn_cache", penalty='graph-net')
decoder.fit(X_train, y_train)
y_pred = decoder.predict(X_test)
accuracy = (y_pred == y_test).mean() * 100.
print("Graph-net classification accuracy : %g%%" % accuracy)

    [NiftiMasker.transform_single_imgs] Resampling images
    [NiftiMasker.transform_single_imgs] Extracting region signals
    [NiftiMasker.transform_single_imgs] Cleaning extracted signals
    Graph-net classification accuracy : 52.9925%

In [ ]:
decoder = SpaceNetClassifier(memory_level=2, n_jobs=4,
                             mask=mask_img, standardize=True,
                             memory="nilearn_cache", penalty='tv-l1')
decoder.fit(X_train, y_train)
y_pred = decoder.predict(X_test)
accuracy = (y_pred == y_test).mean() * 100.
print("TV-l1 classification accuracy : %g%%" % accuracy)

    [NiftiMasker.transform_single_imgs] Resampling images
    [NiftiMasker.transform_single_imgs] Extracting region signals
    [NiftiMasker.transform_single_imgs] Cleaning extracted signals
    TV-l1 classification accuracy : 52.6617%
   